In [ ]:
# You may define any helper functions for Needleman-Wunsch algorithm here
def init_NW(seq1, seq2, M, m ,g):
    # first create an len(seq1 by len(seq2) matrix
    mat = np.array([[0 for i in range(len(seq2)+1)] for j in range(len(seq1)+1)])
    # then populate the leftmost col and upper row
    mat[0] = np.array([g*i for i in range(len(mat[0]))])
    mat[:,0] = np.array([g*i for i in range(len(mat[:, 0]))])
    return (mat)

def score(char1, char2, M, m):
    if char1 == char2:
        return M
    else: return m
   

# need to keep track of backtrack matrix while you're creating the score matrix
def recursive(mat, i, j, str1, str2, seq1, seq2):
    if i == 0 or j ==0:
        if i== 0 and j==0:
            pass    
        elif i == 0:
            str1.append("-"), str2.append(seq2[j-1])
        else: 
            str1.append(seq1[i-1]), str2.append("-")
        return (mat[-1,-1], ("".join(str1))[::-1], ("".join(str2))[::-1])
         
    else:
        if max(mat[i-1, j-1],  mat[i,j-1], mat[i-1, j]) == mat[i-1, j-1]:
            # match or mismatch
            str1.append(seq1[i-1]), str2.append(seq2[j-1])
            return recursive(mat, i-1, j-1, str1, str2, seq1, seq2)
        elif max(mat[i-1, j-1],  mat[i,j-1], mat[i-1, j]) == mat[i, j-1]:
            # add gap to str2
            str1.append("-"), str2.append(seq2[j-1])
            return recursive(mat, i, j-1, str1, str2, seq1, seq2)
        else:
            # add gap to str1
            str1.append(seq1[i-1]), str2.append("-")
            return recursive(mat, i-1, j, str1, str2, seq1, seq2)

def init_backtrack(seq1, seq2):
    # first create an len(seq1 by len(seq2) matrix
    mat = np.array([[0 for i in range(len(seq2)+1)] for j in range(len(seq1)+1)])
    # then populate the leftmost col and upper row
    # 1-- diag, 2--left, 3--up
    mat[0] = np.array([ 2 for i in range(len(mat[0]))])
    mat[:,0] = np.array([3 for i in range(len(mat[:, 0]))])
    mat[0,0] = 0
    return (mat) 

def solve_backtrack(backtrack, seq1, seq2):
    str1, str2 = [], []
    i, j = len(seq1), len(seq2)
    print(i,j)
    while i !=0 or j != 0:
        if backtrack[i,j] == 1:
            str1.append(seq1[i-1]), str2.append(seq2[j-1])
            i -= 1 
            j -= 1
        # left
        elif backtrack[i,j] == 2:
            str1.append("-"), str2.append(seq2[j-1])
            j -= 1
        # up
        elif backtrack[i,j] == 3:
            str1.append(seq1[i-1]), str2.append("-")
            i -= 1
        else:
            break
        #print(i,j)
        #print(str1, str2)
    return ("".join(str1))[::-1], ("".join(str2))[::-1] 
        
        
        
# Do not change this function signature
def needleman_wunsch(seq1, seq2):
    """Find the global alignment for seq1 and seq2
    Returns: 3 items as so:
    the alignment score, alignment in seq1 (str), alignment in seq2 (str)
    """
    # backtrack matrix
    
    backtrack = (init_backtrack(seq1, seq2))
    M, m, g = 1, -2, -1
    mat = init_NW(seq1, seq2, M, m ,g)
    for i in range(1, len(seq1)+1):
        for j in range(1, len(seq2)+1):  
            # now have to partition into if statement
            mat[i, j] = max(mat[i-1, j-1]+score(seq1[i-1], seq2[j-1], M, m), mat[i,j-1]+g, mat[i-1, j]+g)
            # diag
            if mat[i-1, j-1]+score(seq1[i-1], seq2[j-1], M, m) == mat[i,j]:
                backtrack[i,j] = 1
            # left --  this isn't left?
            elif mat[i,j-1]+g == mat[i,j]:
                backtrack[i,j] = 2
            # up
            else:
                backtrack[i,j] = 3
                
    str1, str2 = (solve_backtrack(backtrack, seq1, seq2))
    #str1, str2, i, j  = [], [], len(seq1), len(seq2)
    #return recursive(mat, i, j, str1, str2, seq1, seq2)
    return mat[-1,-1], str1, str2

    # now need to find the best one and traceback
print(needleman_wunsch("AGCTA", "AGGTCA"))     
print(needleman_wunsch("BAF", "AZ"))   

In [2]:
def rle(s):
    """Run Length Encoder
    Args: s, string to be encoded
    Returns: RLE(s)
    If a character is repeated more than once, we replace it with two instances of that 
    character followed by the length of the run. If it only occurs once, we leave it alone
    """
    # only stop and count after letter changes
    out_str = []
    prev = ""
    count = 1
    for i in range(len(s)+1):
        if i == len(s):
            if count > 1:
                out_str.append(prev), out_str.append(prev), out_str.append(str(count))
            else:
                out_str.append(prev)
            return "".join(out_str)
        elif s[i] == prev:
            count += 1
        else:
            if count > 1:
                out_str.append(prev), out_str.append(prev), out_str.append(str(count))
            else:
                out_str.append(prev)
            count = 1
        prev = s[i]
    

#rle("WAAAAHAAAHAAAA!")
len(rle("steelerssteelerssteelerssteelers"))

39

In [4]:
def bwt_encode(s):
    """Burrows-Wheeler Transform
    Args: s, string, which must not contain '{' or '}'
    Returns: BWT(s), which contains '{' and '}'
    """
    """Apply Burrows–Wheeler transform to input string."""
    s = "{" + s + "}"  # Add start and end of text marker
    # given k == n
    # n*n log n
    table = sorted(s[i:] + s[:i] for i in range(len(s)))  # Table of rotations of string
    # n
    last_column = [row[-1:] for row in table]  # Last characters of each row
    return "".join(last_column)  # Convert list of characters into string

def bwt_decode(bwt):
    """Inverse Burrows-Wheeler Transform
    Args: bwt, BWT'ed string, which should contain '{' and '}'
    Returns: reconstructed original string s, must not contains '{' or '}'
    """
    """Apply inverse Burrows–Wheeler transform."""
    r = bwt
    table = [""] * len(r)  # Make empty table
    # k == n
    # n * (n*n log n)
    for i in range(len(r)):
        table = sorted(r[i] + table[i] for i in range(len(r)))  # Add a column of r
    # n
    s = [row for row in table if row.endswith("}")][0]  # Find the correct row (ending in ETX)
    return s.rstrip("}").strip("{")  # Get rid of start and end markers
    
bwt_decode(bwt_encode("peepoop"))

'peepoop'

In [6]:
len(rle(bwt_encode("steelerssteelerssteelerssteelers")))
# the lexicographical sort on the rotations and the repetitive -- long runs of identical characters together

25

In [6]:
print(len(rle(bwt_encode("steal"))), len(("steal")))

7 5


In [7]:
print(len(rle(bwt_encode("aaaaaaaaaa"))), len(("aaaaaaaaaa")))
print(len(rle(bwt_encode("eeeeeee"))), len(("eeeeeee")))
# highly repetitive strings store better
# copy number regions, or tandem repeats

6 10
6 7


In [ ]:
# 7 neighbors? --explain 2^3 -1 (powerset --yada yada) -- crap out what durand says

# hypercube, just keep adding for loops essentially
# O(|N_1| |N_2| ... |N_K|) -- k^2 * 2^k * N^k -- why --again from durand notes

# Yes
# 